Etapa 1: Implementção

kNN:
• Algoritmo para busca do vizinho mais próximo: Força Bruta e KD Tree;
• Métrica: Distância Euclidiana
• k: 1, 3 ou 5 para força bruta, k=1 para KD Tree
• Tipos de Features: Numéricas

In [1]:
import collections

def minkowski(r=2):
    return lambda x,y: sum(abs(x - y)**r) ** (1/r)

def knn(observation, X_treino, Y_treino, k=3, dist=minkowski(2)):
    # dists = lista de pares (distancia, rotulo)
    dists = [ (dist(observation, X_treino[i]), Y_treino[i]) for i in range(len(X_treino)) ]
    dists.sort(key=lambda y: y[1])
    nns = dists[:k]
    # Extrai somente rotulos
    labels = [ n[1] for n in nns ]
    # Retorna lista (de tamanho 1, segundo o parametro) de elemento mais comuns, pares (rotulo, num de occorencias)
    classification = collections.Counter(labels).most_common(1)
    # Retorna apenas o rotulo desse unico elemento
    return classification[0][0]

In [2]:
def knn_kdtree():
    pass

In [3]:
def naive_bayes():
    pass

Etapa 2: Treinamento e Inferência

Primeiramente deve-se organizar os dados e dividí-los entre dados de treino e dados de teste

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = pd.DataFrame(pd.read_csv('../penguins.csv'))


# Shuffle dos dados não balanceados
data = data.sample(frac=1)

# Separar atributo alvo dos restantes
y = data["Species"]
x = data.drop(columns="Species")

# Selecionar apenas colunas numericas relevantes
x = x.drop(columns=["studyName", "Sample Number", "Region", "Island", "Stage", "Individual ID", "Clutch Completion", "Date Egg", "Sex", "Delta 15 N (o/oo)", "Delta 13 C (o/oo)", "Comments"])

# Split dos dados
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.70)

#split = 0.7
#ind_split = int(len(data) * split)
#
#x_treino = x[:ind_split]
#y_treino = y[:ind_split]
#
#x_teste = x[ind_split:]
#y_teste = y[ind_split:]

# Normalizar dados
scaler = StandardScaler()
normalized_train = scaler.fit_transform(x_treino)

normalized_test = scaler.transform(x_teste)

In [13]:
print(normalized_test)

[[-6.53936520e-01 -4.98384243e-02 -5.31646589e-01 -9.90261169e-01]
 [ 8.59402517e-01 -1.59029881e+00  1.38335396e+00  1.39512224e+00]
 [ 9.14101518e-01 -5.11976541e-01  1.52520585e+00  2.27394771e+00]
 [-7.04805059e-02 -1.43625277e+00  1.24150207e+00  5.16296774e-01]
 [-7.81567523e-01  3.09602333e-01 -8.86276321e-01 -1.11580766e+00]
 [ 1.20582953e+00 -7.68719939e-01  1.52520585e+00  1.96008147e+00]
 [-1.78438255e+00 -4.98384243e-02 -1.09905416e+00 -9.90261169e-01]
 [-1.03682953e+00  7.20391770e-01 -7.44424428e-01 -6.13621683e-01]
 [-5.26305517e-01 -2.03884463e-01 -6.73498482e-01 -6.13621683e-01]
 [ 1.20582953e+00 -9.74114657e-01  1.66705775e+00  1.70898848e+00]
 [ 6.95305513e-01  5.66345731e-01 -3.89794697e-01 -4.25301940e-01]
 [ 8.04703516e-01 -8.20068619e-01  1.45427991e+00  1.14402925e+00]
 [ 7.13538513e-01 -1.12816070e+00  1.09965018e+00  7.04616517e-01]
 [ 1.36992653e+00  1.38792460e+00 -1.77016858e-01 -6.13621683e-01]
 [-3.25742512e-01  1.54197064e+00 -7.44424428e-01  7.68840406e

In [15]:
# kd a tree???

# achei a tree
#aux = knn(normalized_test.iloc[[0]],x_teste,y_teste)
#print(np.append(x_teste, y_teste[..., np.newaxis], axis=1))
for (x,y) in zip(normalized_test, y_teste.to_numpy()):
    y_hat = knn(x, normalized_train, y_treino.to_numpy())
    print(y_hat, x)

Adelie Penguin (Pygoscelis adeliae) [-0.65393652 -0.04983842 -0.53164659 -0.99026117]
Adelie Penguin (Pygoscelis adeliae) [ 0.85940252 -1.59029881  1.38335396  1.39512224]
Adelie Penguin (Pygoscelis adeliae) [ 0.91410152 -0.51197654  1.52520585  2.27394771]
Adelie Penguin (Pygoscelis adeliae) [-0.07048051 -1.43625277  1.24150207  0.51629677]
Adelie Penguin (Pygoscelis adeliae) [-0.78156752  0.30960233 -0.88627632 -1.11580766]
Adelie Penguin (Pygoscelis adeliae) [ 1.20582953 -0.76871994  1.52520585  1.96008147]
Adelie Penguin (Pygoscelis adeliae) [-1.78438255 -0.04983842 -1.09905416 -0.99026117]
Adelie Penguin (Pygoscelis adeliae) [-1.03682953  0.72039177 -0.74442443 -0.61362168]
Adelie Penguin (Pygoscelis adeliae) [-0.52630552 -0.20388446 -0.67349848 -0.61362168]
Adelie Penguin (Pygoscelis adeliae) [ 1.20582953 -0.97411466  1.66705775  1.70898848]
Adelie Penguin (Pygoscelis adeliae) [ 0.69530551  0.56634573 -0.3897947  -0.42530194]
Adelie Penguin (Pygoscelis adeliae) [ 0.80470352 -0.82